In [ ]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from openpyxl import load_workbook
import os
import matplotlib.pyplot as plt

In [ ]:
# -------------------------
# 1. Load the data
# -------------------------
file_path = 'Recipe_checked.xlsx'#you can apply your own data of recipes to replace this one
# Read all data
df = pd.read_excel(file_path)

# -------------------------
# 2. Standardize ingredient names (based on similarity >= 80)
# -------------------------
# Get all non-null unique original names
unique_names = df['name'].dropna().unique()

# Build a mapping dictionary: original name -> standardized name
canonical_mapping = {}
canonical_list = []

for name in unique_names:
    found = False
    for canon in canonical_list:
        if fuzz.ratio(name, canon) >= 80:
            canonical_mapping[name] = canon
            found = True
            break
    if not found:
        canonical_list.append(name)
        canonical_mapping[name] = name

# Create a full mapping table
mapping_df = pd.DataFrame(list(canonical_mapping.items()), columns=['original_name', 'unified_name'])

# Apply the mapping to the original data
df['name'] = df['name'].map(canonical_mapping)

# -------------------------
# 3. Output mappings that differ only due to case or pluralization
# -------------------------
def is_trivial_change(original, unified):
    # Return True if they are the same ignoring case
    if original.lower() == unified.lower():
        return True
    # Return True if they are the same after removing trailing 's' (simple plural handling)
    orig = original.lower().rstrip('s')
    unif = unified.lower().rstrip('s')
    return orig == unif

# Filter mappings where names differ only due to case or plural form
trivial_mapping = {orig: unif for orig, unif in canonical_mapping.items() 
                   if orig != unif and is_trivial_change(orig, unif)}
trivial_mapping_df = pd.DataFrame(list(trivial_mapping.items()), columns=['original_name', 'unified_name'])
display(trivial_mapping_df)

In [ ]:
# Save the original 'gram' values to a new column for reference
df['original_gram'] = df['gram']

# -------------------------
# 4. Process weight information
# -------------------------
# Attempt to convert the 'gram' column to numeric values (e.g., "as needed" will be converted to NaN)
df['gram_numeric'] = pd.to_numeric(df['gram'], errors='coerce')

# For each standardized ingredient name, calculate the mean of valid numeric 'gram' values
group_means = df.groupby('name')['gram_numeric'].transform('mean')

def replace_as_needed(row):
    # Replace "as needed" entries with the average weight of the same ingredient
    if isinstance(row['original_gram'], str) and row['original_gram'].strip().lower() == 'as needed':
        return group_means[row.name]
    else:
        return row['gram_numeric']

# Apply replacement to the 'gram' column
df['gram'] = df.apply(replace_as_needed, axis=1)

# Drop the helper column used for numeric conversion
df.drop(columns=['gram_numeric'], inplace=True)

# -------------------------
# 5. Output a check table for replacements of "as needed"
# -------------------------
# Filter records where the original gram value was "as needed"
df_replaced = df[df['original_gram'].astype(str).strip().str.lower() == 'as needed']
# Output a check table including standardized name, original, and replaced gram values
print("\nRecords where 'as needed' was replaced with the average:")
display(df_replaced[['name', 'original_gram', 'gram']])

# -------------------------
# 6. Generate the final DataFrame (keeping all columns, with updated name and gram)
# -------------------------
final_df = df.copy()

display(final_df.head(10))

In [ ]:
# — File paths —
fcd_path = "ASEAN_FCD.xlsx"
input_path = "Recipe_data.xlsx"
output_path = "Nutrition_calculated.xlsx"

# — 1. Load data —
fcd = pd.read_excel(fcd_path, sheet_name="ProcessedTable", dtype={"idnutrition": str})
df = pd.read_excel(input_path, sheet_name="Sheet1", dtype={"idnutrition": str})

# — 2. Use 'gram_adjusted' if available, otherwise fall back to original 'gram' —
df["gram"] = pd.to_numeric(
    df["gram_adjusted"] if "gram_adjusted" in df.columns else df["gram"],
    errors="coerce"
)

# — 2b. Drop entire recipes with missing 'gram' or 'idnutrition' (based on 'index') —
invalid_index = df[df["idnutrition"].isna()]["index"].unique()
removed = df[df["index"].isin(invalid_index)].copy()
df = df[~df["index"].isin(invalid_index)]

# — 3. Merge with nutrition table (keep all rows and track match status) —
df = df.merge(fcd, on="idnutrition", how="left", indicator=True)

# ✅ Save unmatched ingredient records
unmatched = df[df["_merge"] != "both"].copy()
unmatched = unmatched[["index", "recipename", "name", "gram", "idnutrition"]]

# ✅ Keep matched records for nutrient calculations
df = df[df["_merge"] == "both"].drop(columns="_merge")

# — 4. Process nutrient columns —
skip_cols = {"code", "num", "idnutrition", "food_description", "Source"}
nutrients = [col for col in fcd.columns if col not in skip_cols]
for col in nutrients:
    df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)
df[nutrients] = df[nutrients].multiply(df["gram"], axis=0).div(100)

# — 5. Calculate total nutrients per recipe —
total_nutrients = df.groupby(["index", "recipename"], as_index=False)[nutrients].sum()

# — 6. Extract serving size information —
serving_info = (
    df[["index", "recipename", "serving"]]
    .drop_duplicates(subset=["index", "recipename"])
    .assign(serving=lambda d: pd.to_numeric(d["serving"], errors="coerce"))
)

# — 7. Calculate nutrients per serving —
per_serving = total_nutrients.merge(serving_info, on=["index", "recipename"])
for col in nutrients:
    per_serving[col] = per_serving[col] / per_serving["serving"]

# — 8. Ingredient-level nutrient contribution —
ingredient_contribution = df[["index", "recipename", "name", "gram", "idnutrition"] + nutrients]

# — 9. Save results to Excel —
with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    total_nutrients.to_excel(writer, sheet_name="Total_Nutrition", index=False)
    per_serving.to_excel(writer, sheet_name="Per_Serving_Nutrition", index=False)
    ingredient_contribution.to_excel(writer, sheet_name="Ingredient_Nutrient_Contribution", index=False)
    unmatched.to_excel(writer, sheet_name="Unmatched_Ingredients", index=False)
    removed.to_excel(writer, sheet_name="Removed_Recipes", index=False)

In [ ]:
# --- Path settings ---
idea_folder = "/path for IDEA database"
carbon_output_path = "/new file for output"

# Assumption: the DataFrame `df` already contains the following columns:
# ["index", "recipename", "name", "gram", "idnutrition", "IDEA製品コード", "countrycode", "serving"]

# --- 1. Get all unique country codes present in the data ---
countries = df['countrycode'].dropna().unique()
idea_data = {}

# --- 2. Load country-specific IDEA emission intensity data ---
for country in countries:
    idea_file = os.path.join(idea_folder, f"IDEA_v3.3_{country}.xlsx")
    if os.path.exists(idea_file):
        idea_df = pd.read_excel(idea_file, sheet_name="LCIA結果_GWP", header=3, index_col="IDEA製品コード")
        idea_data[country] = idea_df
    else:
        print(f"⚠️ Data for {country} is missing. Will use GLO data as fallback.")

# Load fallback GLO dataset
glo_df = pd.read_excel(os.path.join(idea_folder, "IDEA_v3.3_GLO.xlsx"), 
                       sheet_name="LCIA結果_GWP", header=3, index_col="IDEA製品コード")
glo_map = glo_df['気候変動 IPCC 2021 GWP 100a'].to_dict()

results = []

# --- 3. Calculate carbon footprint per recipe, grouped by country ---
for country, group_df in df.groupby('countrycode'):
    group_df = group_df.copy()
    display(group_df)
    idea_df = idea_data.get(country, glo_df)
    country_suffix = country if country in idea_data else 'GLO'
    
    # Modify IDEA product code to match country-specific format
    group_df['IDEA製品コード_国'] = group_df['IDEA製品コード'].apply(
        lambda x: x[:-3] + country_suffix if isinstance(x, str) and len(x) >= 3 else x
    )
    
    # Merge with emission intensity data
    group_df = group_df.merge(
        idea_df[['気候変動 IPCC 2021 GWP 100a']],
        left_on='IDEA製品コード_国',
        right_index=True,
        how='left'
    )
    
    # Fill unmatched records with fallback GLO values
    unmatched = group_df['気候変動 IPCC 2021 GWP 100a'].isna()
    if unmatched.any():
        print(f"⚠️ {country}: {unmatched.sum()} unmatched records. Using GLO as fallback.")
        group_df.loc[unmatched, 'IDEA製品コード_GLO'] = group_df.loc[unmatched, 'IDEA製品コード'].apply(
            lambda x: x[:-3] + 'GLO' if pd.notnull(x) else x
        )
        group_df.loc[unmatched, '気候変動 IPCC 2021 GWP 100a'] = group_df.loc[unmatched, 'IDEA製品コード_GLO'].map(glo_map)
    
    # Report any entries still unmatched after fallback
    if group_df['気候変動 IPCC 2021 GWP 100a'].isna().any():
        unmatched_final = group_df[group_df['気候変動 IPCC 2021 GWP 100a'].isna()][['name', 'IDEA製品コード']]
        print(f"❌ Unmatched records remain in {country}:")
        display(unmatched_final)

    # Calculate ingredient-level carbon footprint (in kg CO2-eq)
    group_df['ingredient_footprint'] = (group_df['gram'] / 1000) * group_df['気候変動 IPCC 2021 GWP 100a'].fillna(0)

    # Aggregate carbon footprint at the recipe level
    dish_footprint = group_df.groupby(['index', 'recipename'], as_index=False).agg({
        'ingredient_footprint': 'sum',
        'serving': 'first',
        'countrycode': 'first'
    })
    
    # Compute footprint per serving
    dish_footprint['footprint_per_serving'] = dish_footprint['ingredient_footprint'] / dish_footprint['serving']
    results.append(dish_footprint)

# --- 4. Combine results from all countries ---
final_footprint = pd.concat(results, ignore_index=True)

# --- 5. Export to Excel ---
final_footprint.to_excel(carbon_output_path, index=False)

In [ ]:
# Read the Excel file
file_path = "Recipe_Data.xlsx"
df = pd.read_excel(file_path)

# Count the number of dishes by country and dish type
country_dish_counts = df.groupby(['country', 'dish type 1']).size().unstack(fill_value=0)

# Output the statistics
print("Number of each dish type per country:")
print(country_dish_counts)

# Use the provided color codes
color_map = {
    'beef': '#E3A967',
    'pork': '#EEC97C',
    'chicken': '#F7E4BA',
    'seafood': '#B0D6D9',
    'vegetable': '#7FB2C9',
    'other': '#5D88A3'
}

# Define a fixed dish type order to keep pie chart colors consistent
dish_order = ['beef', 'pork', 'chicken', 'seafood', 'vegetable', 'other']

# Plot a pie chart for each country
for country, row in country_dish_counts.iterrows():
    row = row.reindex(dish_order, fill_value=0)
    row_nonzero = row[row > 0]  # Only include dish types with non-zero values
    fig, ax = plt.subplots()
    ax.pie(
        row_nonzero,
        colors=[color_map[d] for d in row_nonzero.index],
        startangle=90  # Start from top to make layout cleaner
    )
    plt.axis('equal')  # Ensure the pie chart is a circle
    plt.tight_layout()
    plt.savefig(f"{country}_dish_type_1_pie.svg", format='svg', transparent=True)
    plt.close()

In [ ]:
# Count the number of dishes by country and dish type 2
country_dish_counts2 = df.groupby(['country', 'dish type 2']).size().unstack(fill_value=0)

# Define the order of dish categories
dish_order2 = [
    'Beverages/Drinks',
    'Condiments/Seasonings/Sauces',
    'Meat Dishes',
    'Noodle-Based Dishes',
    'Others',
    'Salads',
    'Seafood Dishes',
    'Side Dishes/Pickles',
    'Snacks/Desserts',
    'Soups',
    'Vegetarian Dishes'
]

# Define a custom spectral color palette
custom_colors = [
    '#9e0142', '#d53e4f', '#f46d43', '#fdae61', '#fee08b',
    '#ffffbf', '#e6f598', '#abdda4', '#66c2a5', '#3288bd', '#5e4fa2'
]

# Create a color mapping dictionary for dish types
color_map2 = dict(zip(dish_order2, custom_colors))

# Define the output directory for saving pie charts
save_dir = "/Pie chart"
os.makedirs(save_dir, exist_ok=True)

# Generate and save pie charts for each country's dish type 2 distribution
for country, row in country_dish_counts2.iterrows():
    row = row.reindex(dish_order2, fill_value=0)
    row_nonzero = row[row > 0]
    if row_nonzero.empty:
        continue
    
    fig, ax = plt.subplots()
    ax.pie(
        row_nonzero,
        labels=None,  # Do not display labels
        colors=[color_map2[d] for d in row_nonzero.index],
        startangle=90,
        autopct=None  # Do not display percentages
    )

#     ax.set_title(country, fontsize=12)  # Optional: add country name as title
    plt.axis('equal')  # Keep pie chart circular
    plt.tight_layout()
    
    out_path = os.path.join(save_dir, f"{country}_dish_type_2_pie.svg")
    plt.savefig(out_path, format='svg', transparent=True)
    plt.close(fig)